In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
 
import tensorflow as tf
import pathlib
import random
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/DKuzn/Recognition_RusHandRilNet.git

fatal: destination path 'Recognition_RusHandRilNet' already exists and is not an empty directory.


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 100

In [ ]:
data_root_orig_train = '/content/Recognition_RusHandRilNet/Train'
data_root_train = pathlib.Path(data_root_orig_train)
data_root_orig_test = '/content/Recognition_RusHandRilNet/Test'
data_root_test = pathlib.Path(data_root_orig_test)

In [ ]:
train_image_paths = list(data_root_train.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

image_count = len(train_image_paths)

In [ ]:
test_image_paths = list(data_root_test.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
random.shuffle(test_image_paths)

image_count = len(test_image_paths)

In [ ]:
train_image_paths = list(data_root_train.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

In [ ]:
test_image_paths = list(data_root_test.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
random.shuffle(test_image_paths)

In [ ]:
train_label_names = sorted(item.name for item in data_root_train.glob('*/') if item.is_dir())
test_label_names = sorted(item.name for item in data_root_test.glob('*/') if item.is_dir())
train_label_to_index = dict((name, index) for index, name in enumerate(train_label_names))
test_label_to_index = dict((name, index) for index, name in enumerate(test_label_names))


In [ ]:
train_image_labels = [train_label_to_index[pathlib.Path(path).parent.name]
                    for path in train_image_paths]


In [ ]:
test_image_labels = [test_label_to_index[pathlib.Path(path).parent.name]
                    for path in test_image_paths]


In [ ]:
def preprocess_image(image):
    image = tf.io.decode_jpeg(image, channels=1)
    image = tf.image.resize(image, [28, 28])
    image /= 255.0

    return image

In [ ]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [ ]:
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)

train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

train_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_image_labels, tf.int64))
test_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(test_image_labels, tf.int64))

train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))

In [ ]:
train_ds = train_image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
test_ds = test_image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=62, 
                                 kernel_size=(3, 3), 
                                 activation='relu', 
                                 padding='valid', 
                                 input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(filters=62, 
                                 kernel_size=(3, 3), 
                                 activation='relu', 
                                 padding='valid'))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=124, 
                                 kernel_size=(3, 3), 
                                 activation='relu', 
                                 padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=124, 
                                 kernel_size=(3, 3), 
                                 activation='relu', 
                                 padding='valid'))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Conv2D(filters=248, 
                                 kernel_size=(3, 3), 
                                 activation='relu', 
                                 padding='valid'))
model.add(tf.keras.layers.MaxPool2D((2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(124, activation='relu'))
model.add(tf.keras.layers.Dense(62))

In [ ]:
len(model.trainable_variables)

14

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 26, 26, 62)        620       
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 24, 24, 62)        34658     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 12, 12, 62)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 10, 10, 124)       69316     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 8, 8, 124)         138508    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 4, 4, 124)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 2, 2, 248)        

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
steps_per_epoch=tf.math.ceil(len(train_image_paths)/BATCH_SIZE).numpy()
steps_per_epoch

2325.0

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(train_ds, epochs=100, steps_per_epoch=2325, validation_data=test_ds, validation_steps=930)

In [ ]:
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# model.save('Recognition_RusHandRilNet_new_arch_v3.h5')